In [1]:
import pandas as pd
import os
from pathlib import Path
import re
from wordcloud import WordCloud
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import gensim.corpora as corpora
from pprint import pprint
import pyLDAvis.gensim
import pickle
import pyLDAvis
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from datasets.load import requests
from pypdf import PdfReader

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alejandro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-04-22 19:03:52,762 : INFO : PyTorch version 2.2.1 available.


In [ ]:
def extract_documents_from_pdf(url):

    output_path = os.path.join(static_dir, 'text') + "/temp"
    print(url)

    response = requests.get(url)
    contentType = response.headers.get('content-type')
    if 'pdf' in contentType.lower():
        with open(url, 'rb') as f:
            pdf_reader = PdfReader(f)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                yield page.extract_text()
    elif 'txt' in contentType.lower():
        with open(url) as src:
            print(src.readline()[0])
            yield src.readlines()[0]
url = 'https://procrastinatingbucket.sgp1.digitaloceanspaces.com/Procrastinate/Procrastinate jane12/speechToText/529e59c9'
docs = list(extract_documents_from_pdf(url))

In [2]:
def extract_documents_from_pdf(url):
    # response = requests.get(url)
    # print(response.headers.get('content-type'))
    # with open('temp_pdf.pdf', 'wb') as f:
    #     f.write(response.content)

    with open('/home/alejandro/Downloads/Animal Farm Full Text.pdf', 'rb') as f:
        pdf_reader = PdfReader(f)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            yield page.extract_text()
            
url = 'https://procrastinatingbucket.sgp1.digitaloceanspaces.com/Procrastinate/Procrastinate jane12/speechToText/529e59c9'
docs = list(extract_documents_from_pdf(url))

In [3]:

cleaned_docs = []

for doc in docs:
    punc_removed = re.sub('[,\.!?]', '', doc)
    case_lowered = punc_removed.lower()
    fillers_removed_doc = re.sub(
        r'\b(?:okay|right|going|thing|kind|really|thing|one|lot|see|look|might|actually|now|different|pretty|think|kind|maybe|want|need|understand|well|like|little|could|say|also|all|can|will|welcome|our|first|lecture|to|up|try|broadly|where|is|about|the|just|do|get)\b',
        '',
        case_lowered
    )
    
    
    cleaned_docs.append(fillers_removed_doc)
    
docs = cleaned_docs
print(docs)

['animal farm\ngeorge orwell\n1945', 'i\nmr jones of  manor farm had locked  hen-houses for  night but\nwas too drunk  remember  shut  popholes with  ring of light from\nhis lantern dancing from side  side he lurched across  yard kicked oﬀ his\nboots at  back door drew himself a last glass of beer from  barrel in \nscullery and made his way   bed  mrs jones was already snoring\nas soon as  light in  bedroom went out there was a stirring and a\nﬂuttering  through  farm buildings word had gone round during  day\nthat old major  prize middle white boar had had a strange dream on \nprevious night and wished  communicate it   other animals it had been\nagreed that they should  meet in  big barn as soon as mr jones was safely\nout of  way old major (so he was always called though  name under\nwhich he had been exhibited was willingdon beauty) was so highly regarded on\n farm that everyone was quite ready  lose an hour’s sleep in order  hear\nwhat he had  \nat  end of  big barn on a sort of r

<>:4: DeprecationWarning: invalid escape sequence '\.'
<>:4: DeprecationWarning: invalid escape sequence '\.'
/tmp/ipykernel_172240/55492016.py:4: DeprecationWarning: invalid escape sequence '\.'
  punc_removed = re.sub('[,\.!?]', '', doc)


In [4]:
def tokeniz(sentences):

    for sentence in sentences:

        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

processed_data = list(tokeniz(docs))
print(processed_data)

[['animal', 'farm', 'george', 'orwell'], ['mr', 'jones', 'of', 'manor', 'farm', 'had', 'locked', 'hen', 'houses', 'for', 'night', 'but', 'was', 'too', 'drunk', 'remember', 'shut', 'popholes', 'with', 'ring', 'of', 'light', 'from', 'his', 'lantern', 'dancing', 'from', 'side', 'side', 'he', 'lurched', 'across', 'yard', 'kicked', 'oﬀ', 'his', 'boots', 'at', 'back', 'door', 'drew', 'himself', 'last', 'glass', 'of', 'beer', 'from', 'barrel', 'in', 'scullery', 'and', 'made', 'his', 'way', 'bed', 'mrs', 'jones', 'was', 'already', 'snoring', 'as', 'soon', 'as', 'light', 'in', 'bedroom', 'went', 'out', 'there', 'was', 'stirring', 'and', 'ﬂuttering', 'through', 'farm', 'buildings', 'word', 'had', 'gone', 'round', 'during', 'day', 'that', 'old', 'major', 'prize', 'middle', 'white', 'boar', 'had', 'had', 'strange', 'dream', 'on', 'previous', 'night', 'and', 'wished', 'communicate', 'it', 'other', 'animals', 'it', 'had', 'been', 'agreed', 'that', 'they', 'should', 'meet', 'in', 'big', 'barn', 'as',

In [5]:
def remove_stopwords(texts):

    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatize(texts, allowed_postags=['NOUN', 'ADJ', 'VERB']):

    texts_op = []

    for sent in texts:

        doc = nlp(" ".join(sent))

        texts_op.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])

    return texts_op

In [6]:

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','for'])
data_wo_stopwords = remove_stopwords(processed_data)
print(data_wo_stopwords)

[['animal', 'farm', 'george', 'orwell'], ['mr', 'jones', 'manor', 'farm', 'locked', 'hen', 'houses', 'night', 'drunk', 'remember', 'shut', 'popholes', 'ring', 'light', 'lantern', 'dancing', 'side', 'side', 'lurched', 'across', 'yard', 'kicked', 'oﬀ', 'boots', 'back', 'door', 'drew', 'last', 'glass', 'beer', 'barrel', 'scullery', 'made', 'way', 'bed', 'mrs', 'jones', 'already', 'snoring', 'soon', 'light', 'bedroom', 'went', 'stirring', 'ﬂuttering', 'farm', 'buildings', 'word', 'gone', 'round', 'day', 'old', 'major', 'prize', 'middle', 'white', 'boar', 'strange', 'dream', 'previous', 'night', 'wished', 'communicate', 'animals', 'agreed', 'meet', 'big', 'barn', 'soon', 'mr', 'jones', 'safely', 'way', 'old', 'major', 'always', 'called', 'though', 'name', 'exhibited', 'willingdon', 'beauty', 'highly', 'regarded', 'farm', 'everyone', 'quite', 'ready', 'lose', 'hour', 'sleep', 'order', 'hear', 'end', 'big', 'barn', 'sort', 'raised', 'platform', 'major', 'already', 'ensconced', 'bed', 'straw',

In [7]:
bigram = gensim.models.Phrases(processed_data, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

def create_bigrams(texts):

    return [bigram_mod[doc] for doc in texts]

2024-04-22 19:03:53,444 : INFO : collecting all words and their counts
2024-04-22 19:03:53,445 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-04-22 19:03:53,458 : INFO : collected 21509 token types (unigram + bigrams) from a corpus of 25408 words and 55 sentences
2024-04-22 19:03:53,458 : INFO : merged Phrases<21509 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
2024-04-22 19:03:53,458 : INFO : Phrases lifecycle event {'msg': 'built Phrases<21509 vocab, min_count=5, threshold=100, max_vocab_size=40000000> in 0.01s', 'datetime': '2024-04-22T19:03:53.458907', 'gensim': '4.3.2', 'python': '3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]', 'platform': 'Linux-6.5.0-10031-tuxedo-x86_64-with-glibc2.35', 'event': 'created'}
2024-04-22 19:03:53,459 : INFO : exporting phrases from Phrases<21509 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
2024-04-22 19:03:53,474 : INFO : FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<13 ph

In [8]:
data_bigrams = create_bigrams(data_wo_stopwords)

In [9]:
import spacy
nlp=spacy.load('en_core_web_sm',disable=['parser', 'ner'])
data_lemmatized = lemmatize(data_bigrams, allowed_postags=[ 'NOUN', 'ADJ', 'VERB'])

In [10]:
gensim_dictionary = corpora.Dictionary(data_lemmatized)

2024-04-22 19:03:55,365 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-04-22 19:03:55,372 : INFO : built Dictionary<2581 unique tokens: ['animal', 'admit', 'agree', 'appearance', 'approach']...> from 55 documents (total 10997 corpus positions)
2024-04-22 19:03:55,373 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<2581 unique tokens: ['animal', 'admit', 'agree', 'appearance', 'approach']...> from 55 documents (total 10997 corpus positions)", 'datetime': '2024-04-22T19:03:55.373738', 'gensim': '4.3.2', 'python': '3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]', 'platform': 'Linux-6.5.0-10031-tuxedo-x86_64-with-glibc2.35', 'event': 'created'}


In [11]:
texts = data_lemmatized

In [12]:
gensim_corpus = [gensim_dictionary.doc2bow(text) for text in texts]

In [13]:
[[(gensim_dictionary[id], freq) for id, freq in cp] for cp in gensim_corpus[:4]] 

[[('animal', 1)],
 [('animal', 5),
  ('admit', 1),
  ('agree', 1),
  ('appearance', 2),
  ('approach', 1),
  ('arrive', 1),
  ('ask', 1),
  ('bad', 1),
  ('barrel', 1),
  ('beast', 1),
  ('beauty', 1),
  ('bed', 2),
  ('bedroom', 1),
  ('beer', 1),
  ('begin', 2),
  ('benevolent', 1),
  ('benjamin', 1),
  ('big_barn', 1),
  ('bluebell', 1),
  ('boot', 1),
  ('boxer', 3),
  ('building', 1),
  ('call', 1),
  ('care', 1),
  ('cart', 1),
  ('character', 1),
  ('clover', 2),
  ('come', 3),
  ('comfortable', 1),
  ('communicate', 1),
  ('conceal', 1),
  ('cow', 1),
  ('cut', 1),
  ('cynical', 1),
  ('dancing', 1),
  ('day', 1),
  ('devoted', 1),
  ('diﬀerent', 1),
  ('dog', 1),
  ('door', 1),
  ('draw', 1),
  ('dream', 1),
  ('drunk', 1),
  ('end', 1),
  ('enormous', 1),
  ('ensconce', 1),
  ('exhibit', 1),
  ('fact', 2),
  ('farm', 5),
  ('fashion', 1),
  ('foal', 1),
  ('fourth', 1),
  ('front', 1),
  ('get', 1),
  ('give', 2),
  ('glass', 1),
  ('go', 2),
  ('great', 1),
  ('grow', 1),
  

In [14]:
lda_model = gensim.models.ldamodel.LdaModel(

   corpus=gensim_corpus, id2word=gensim_dictionary, num_topics=15, random_state=100, 

   update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True

)

2024-04-22 19:03:55,452 : INFO : using autotuned alpha, starting with [0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667]
2024-04-22 19:03:55,453 : INFO : using symmetric eta at 0.06666666666666667
2024-04-22 19:03:55,454 : INFO : using serial LDA version on this node
2024-04-22 19:03:55,457 : INFO : running online (multi-pass) LDA training, 15 topics, 10 passes over the supplied corpus of 55 documents, updating model once every 55 documents, evaluating perplexity every 55 documents, iterating 50x with a convergence threshold of 0.001000
2024-04-22 19:03:55,487 : INFO : -13.518 per-word bound, 11730.8 perplexity estimate based on a held-out corpus of 55 documents with 10997 words
2024-04-22 19:03:55,487 : INFO : PROGRESS: pass 0, at document #55/55
2024-04-22 19:03:55,505 : INFO : optimized alpha [0.07970283, 0.08875321, 0.09529638, 0.11160034, 0.07636237, 0.

In [15]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(

   model=lda_model, texts=data_lemmatized, dictionary=gensim_dictionary, coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()

print('\nCoherence Score: ', coherence_lda)

2024-04-22 19:03:55,867 : INFO : using ParallelWordOccurrenceAccumulator<processes=19, batch_size=64> to estimate probabilities from sliding windows
2024-04-22 19:03:56,071 : INFO : 1 batches submitted to accumulate stats from 64 documents (5002 virtual)
2024-04-22 19:03:56,074 : INFO : serializing accumulator to return to master...
2024-04-22 19:03:56,074 : INFO : serializing accumulator to return to master...
2024-04-22 19:03:56,074 : INFO : serializing accumulator to return to master...
2024-04-22 19:03:56,074 : INFO : serializing accumulator to return to master...
2024-04-22 19:03:56,075 : INFO : serializing accumulator to return to master...
2024-04-22 19:03:56,074 : INFO : serializing accumulator to return to master...
2024-04-22 19:03:56,075 : INFO : serializing accumulator to return to master...
2024-04-22 19:03:56,075 : INFO : serializing accumulator to return to master...
2024-04-22 19:03:56,075 : INFO : serializing accumulator to return to master...
2024-04-22 19:03:56,075 :


Coherence Score:  0.30040316872742157


In [16]:
top_topics = lda_model.top_topics(gensim_corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / 15
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -1.0759.
[([(0.0010570263, 'animal'),
   (0.0006684854, 'pig'),
   (0.0006348481, 'farm'),
   (0.0006220652, 'snowball'),
   (0.0006195989, 'time'),
   (0.00060547265, 'say'),
   (0.00060068304, 'boxer'),
   (0.00057636417, 'come'),
   (0.00056281814, 'go'),
   (0.00055264717, 'make'),
   (0.0005422047, 'comrade'),
   (0.00053921435, 'work'),
   (0.0005326676, 'squealer'),
   (0.0005313741, 'day'),
   (0.0005228543, 'give'),
   (0.0005221717, 'man'),
   (0.0005160941, 'take'),
   (0.0005071708, 'moment'),
   (0.0005040758, 'windmill'),
   (0.00049769436, 'dog')],
  -0.5616131689452329),
 ([(0.03216429, 'animal'),
   (0.016860833, 'farm'),
   (0.012512546, 'snowball'),
   (0.009982065, 'work'),
   (0.009962998, 'time'),
   (0.00850358, 'say'),
   (0.008033231, 'come'),
   (0.008008659, 'day'),
   (0.0076136547, 'pig'),
   (0.0065429104, 'squealer'),
   (0.00604885, 'take'),
   (0.005853654, 'napoleon'),
   (0.005630386, 'windmill'),
   (0.005555656, 'go'),
   (0

In [17]:
# Find the topic with the highest coherence score
top_top_word = []

seen_words = set()
for topic_words, coherence_score in top_topics:
    # Get the word with the highest probability in the topic
    top_word = max(topic_words, key=lambda x: x[0])
    print(top_word)
    
    if top_word[1] not in seen_words:
        top_top_word.append(top_word)
        seen_words.add(top_word[1]) 
        
# Print the top word and its probability
    print("Top Word in Topic:")
    print("Word:", top_word[1])
    print("Coherence Score:", top_word[0])
    print()
print(top_top_word)
print(seen_words)

(0.0010570263, 'animal')
Top Word in Topic:
Word: animal
Coherence Score: 0.0010570263

(0.03216429, 'animal')
Top Word in Topic:
Word: animal
Coherence Score: 0.03216429

(0.02646949, 'animal')
Top Word in Topic:
Word: animal
Coherence Score: 0.02646949

(0.02270818, 'animal')
Top Word in Topic:
Word: animal
Coherence Score: 0.02270818

(0.030532897, 'animal')
Top Word in Topic:
Word: animal
Coherence Score: 0.030532897

(0.00060071994, 'animal')
Top Word in Topic:
Word: animal
Coherence Score: 0.00060071994

(0.020197107, 'animal')
Top Word in Topic:
Word: animal
Coherence Score: 0.020197107

(0.018906713, 'work')
Top Word in Topic:
Word: work
Coherence Score: 0.018906713

(0.017389664, 'animal')
Top Word in Topic:
Word: animal
Coherence Score: 0.017389664

(0.039080467, 'animal')
Top Word in Topic:
Word: animal
Coherence Score: 0.039080467

(0.028503653, 'snowball')
Top Word in Topic:
Word: snowball
Coherence Score: 0.028503653

(0.012132311, 'life')
Top Word in Topic:
Word: life
Co

In [18]:
# Sort the subjects based on coherence scores
sorted_subjects = sorted(top_top_word, key=lambda x: x[0], reverse=True)
print(sorted_subjects)

# Get the top 3 subjects
top_5_subjects = sorted_subjects[:5]

# Print the top 3 subjects
for i, (coherence_score,subject) in enumerate(top_5_subjects, 1):
    print(f"Top Subject {i}:")
    print("Subject:", subject)
    print("Coherence Score:", coherence_score)
    print()

[(0.028503653, 'snowball'), (0.018906713, 'work'), (0.012416087, 'farm'), (0.012132311, 'life'), (0.0010570263, 'animal')]
Top Subject 1:
Subject: snowball
Coherence Score: 0.028503653

Top Subject 2:
Subject: work
Coherence Score: 0.018906713

Top Subject 3:
Subject: farm
Coherence Score: 0.012416087

Top Subject 4:
Subject: life
Coherence Score: 0.012132311

Top Subject 5:
Subject: animal
Coherence Score: 0.0010570263

